In [1]:
import torch
import torch.nn as nn
import numpy as np
import lightning as pl

from torch.utils.data import DataLoader,TensorDataset
from torch.optim import RMSprop
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

d:\New folder\Project\ProjectData\Data Science\datascience\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
features,target = make_classification(n_classes=2,
                                      n_features=10,
                                      n_samples=1000)

features_train,features_test,target_train,target_test=train_test_split(features,
                                                                       target,
                                                                       test_size=0.1,
                                                                       random_state=1)

torch.manual_seed(0)
np.random.seed(0)

x_train = torch.from_numpy(features_train).float()
y_train = torch.from_numpy(target_train).float().view(-1,1)
x_test = torch.from_numpy(features_test).float()
y_test = torch.from_numpy(target_test).float().view(-1,1)

In [3]:
class SimpleNeuralNet(nn.Module):
    def __init__(self):
        super(SimpleNeuralNet,self).__init__()
        self.sequential = torch.nn.Sequential(
            torch.nn.Linear(10,16),
            torch.nn.ReLU(),
            torch.nn.Linear(16,16),
            torch.nn.ReLU(),
            torch.nn.Linear(16,1),
            torch.nn.Sigmoid()
        )
    
    def forward(self,x):
        x =self.sequential(x)
        return(x)

In [4]:
class LightningNetwork(pl.LightningModule):
    def __init__(self,network):
        super().__init__()
        self.network = network
        self.criterion = nn.BCELoss()
        self.metric = nn.functional.binary_cross_entropy

    def training_step(self, batch, batch_idx):
        data,target = batch
        output = self.network(data)
        loss = self.criterion(output,target)
        self.log("val_loss",loss)
        return loss
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(),lr=1e-3)

In [5]:
train_data = TensorDataset(x_train,y_train)
train_loader = DataLoader(train_data,batch_size=100,shuffle=True)

network = LightningNetwork(SimpleNeuralNet())

trainer = pl.Trainer(callbacks=[EarlyStopping(monitor="val_loss",mode="min",patience=3)],max_epochs=1000)

trainer.fit(model=network,train_dataloaders=train_loader)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name      | Type            | Params | Mode 
------------------------------------------------------
0 | network   | SimpleNeuralNet | 465    | train
1 | criterion | BCELoss         | 0      | train
------------------------------------------------------
465       Trainable params
0         Non-trainable params
465       Total params
0.002     Total estimated model params size (MB)
9         Modules in train mode
0         Modules in eval mode
d:\New folder\Project\ProjectData\Data Science\datascience\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:433: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argu

Epoch 13: 100%|██████████| 9/9 [00:00<00:00, 137.02it/s, v_num=1]


Discussion

In [7]:
trainer = pl.Trainer(max_epochs=1000)
trainer.fit(model=network,train_dataloaders=train_loader)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name      | Type            | Params | Mode 
------------------------------------------------------
0 | network   | SimpleNeuralNet | 465    | train
1 | criterion | BCELoss         | 0      | train
------------------------------------------------------
465       Trainable params
0         Non-trainable params
465       Total params
0.002     Total estimated model params size (MB)
9         Modules in train mode
0         Modules in eval mode
d:\New folder\Project\ProjectData\Data Science\datascience\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:433: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argu

Epoch 999: 100%|██████████| 9/9 [00:00<00:00, 48.22it/s, v_num=2] 

`Trainer.fit` stopped: `max_epochs=1000` reached.


Epoch 999: 100%|██████████| 9/9 [00:00<00:00, 46.24it/s, v_num=2]
